## Topic discovery: yelp review

This notebook shows how to load a huge LLM and perform inference using ```accelerate```. See ```0_guide``` for detail and reference.

In [1]:
# from huggingface_hub import hf_hub_download 
import torch
from datasets import load_dataset, load_from_disk
import evaluate
import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from transformers import MistralForCausalLM
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, TaskType, get_peft_model
from peft import PeftConfig, PeftModel
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from huggingface_hub import notebook_login
from accelerate import init_empty_weights, load_checkpoint_and_dispatch, infer_auto_device_map, dispatch_model

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import re
import os
import glob

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string

D:\cygdrive\d\projects\LLM_py38\venv\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
D:\cygdrive\d\projects\LLM_py38\venv\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
D:\cygdrive\d\projects\LLM_py38\venv\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
print(transformers.__version__)

4.35.2


In [3]:
print(torch.__version__)
print(torch.cuda.is_available())

2.3.0+cu118
True


In [4]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [6]:
llm_repo_dir = 'D:/projects/LLM'
cache_dir = '/cygdrive/d/projects/LLM/.cache'
os.environ['TRANSFORMERS_CACHE'] = cache_dir
os.environ['HF_HOME'] = cache_dir + '/huggingface'
os.environ['XDG_CACHE_HOME'] = cache_dir
os.environ['HF_DATASETS_CACHE'] = cache_dir

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
dataset_path = cache_dir + '/parquet/yelp_polarity' # cache_dir + '/parquet/yelp_review_full-e22176106d6e7534'
dataset_name = 'yelp_polarity' # yelp_review_full

if not os.path.isdir(dataset_path):
    dataset = load_dataset(dataset_name, cache_dir=cache_dir + '/parquet')
else:
    dataset = load_dataset(dataset_path)

In [8]:
tokenized_data_path = cache_dir + '/tokenized_dataset_yelp_polarity_bert'

tokenized_datasets = load_from_disk(tokenized_data_path)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [10]:
print(type(small_train_dataset))
print(small_train_dataset.shape)

<class 'datasets.arrow_dataset.Dataset'>
(1000, 5)


## Gen AI

In [9]:
## T5-3b
#model_name = 'google-t5/t5-3b'
#model_path = cache_dir + '/models--google-t5--t5-3b/snapshots/bed96aab9ee46012a5046386105ee5fd0ac572f0'

# Mistral 7B
model_name = 'mistralai/Mistral-7B-v0.1'
model_path = cache_dir + '/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24'

In [10]:
# need this to get token to access a gated model
# notebook_login()

In [10]:
mistral_tokenizer = AutoTokenizer.from_pretrained(model_path, 
                                          cache_dir=cache_dir, 
                                          local_flies_only=True)

# load empty model to save memory
with init_empty_weights():
    mistral_model = AutoModelForCausalLM.from_pretrained(model_path, 
                                             cache_dir=cache_dir,
                                             local_files_only=True,
                                             load_in_8bit=True)
    
mistral_model = load_checkpoint_and_dispatch(mistral_model, 
                                             model_path,
                                             device_map='auto',
                                             offload_folder=None)

device_map  = infer_auto_device_map(mistral_model.model)
full_model_device_map = {f"model.{k}": v for k, v in device_map.items()}
full_model_device_map["lm_head"] = 0
dispatch_model(mistral_model, device_map=full_model_device_map) # make sure on the same device

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

D:\cygdrive\d\projects\LLM_py38\venv\lib\site-packages\accelerate\utils\modeling.py:1363: UserWarning: Current model requires 536879104 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
D:\cygdrive\d\projects\LLM_py38\venv\lib\site-packages\accelerate\utils\modeling.py:1363: UserWarning: Current model requires 268439552 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMS

In [11]:
dataset['test']['text'][50]

'Our experience not so great.  We have mortgage as well as basic free checking and savings with Dollar in Squirrel Hill.  Mortgage application process was brutal.  Starting with the advertised rate of 2.85% being suddenly not available when we decided to lock it in; got 3.01%.  Automatic deduction of mortgage payment was required to secure this rate and it came out of the required Dollar checking account on the first business day of the month.  Free checking is free ... until.  Beware of Funds Availability policy.  We deposit a check monthly at the end of the month.  In August, the end of the month was a weekend and Labor Day was on September 1.  So our check did not get deposited until 9/2.  Mortgage payment hit on 9/2. It was paid, but we were charged $36 insufficient funds due solely to Funds Availability Policy.  After doing this first, Dollar then merrily hits us with two more $36 charges on smaller checks that would have cleared had they been processed before the mortgage payment

In [12]:
prompt = """ 
Describe the theme of the comment in one word.

Comment: {0}
Theme: healthcare

Comment: {1}
Theme: food

Comment: {2}
Theme: recreational

Comment: {3}
Theme: electronics

Comment: {4}
Theme:
""".format(dataset['train']['text'][0].strip(),
           dataset['train']['text'][4].strip(),
           dataset['train']['text'][8].strip(),
           dataset['train']['text'][10].strip(),
           dataset['test']['text'][50].strip())

In [18]:
prompt = """
What topic describes the following comment:
{0}
""".format(dataset['test']['text'][50].strip()).strip()

In [21]:
prompt

'What topic describes the following comment:\nOur experience not so great.  We have mortgage as well as basic free checking and savings with Dollar in Squirrel Hill.  Mortgage application process was brutal.  Starting with the advertised rate of 2.85% being suddenly not available when we decided to lock it in; got 3.01%.  Automatic deduction of mortgage payment was required to secure this rate and it came out of the required Dollar checking account on the first business day of the month.  Free checking is free ... until.  Beware of Funds Availability policy.  We deposit a check monthly at the end of the month.  In August, the end of the month was a weekend and Labor Day was on September 1.  So our check did not get deposited until 9/2.  Mortgage payment hit on 9/2. It was paid, but we were charged $36 insufficient funds due solely to Funds Availability Policy.  After doing this first, Dollar then merrily hits us with two more $36 charges on smaller checks that would have cleared had th

In [22]:
%%time

input_ids = mistral_tokenizer(prompt, return_tensors="pt").input_ids
with torch.no_grad():
    generated_ids = mistral_model.generate(input_ids, 
                                      do_sample=True,
                                      temperature=0.1,
                                      top_k=10,
                                      top_p=10,
                                      max_new_tokens=3)
    generated_text = mistral_tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


What topic describes the following comment:
Our experience not so great.  We have mortgage as well as basic free checking and savings with Dollar in Squirrel Hill.  Mortgage application process was brutal.  Starting with the advertised rate of 2.85% being suddenly not available when we decided to lock it in; got 3.01%.  Automatic deduction of mortgage payment was required to secure this rate and it came out of the required Dollar checking account on the first business day of the month.  Free checking is free ... until.  Beware of Funds Availability policy.  We deposit a check monthly at the end of the month.  In August, the end of the month was a weekend and Labor Day was on September 1.  So our check did not get deposited until 9/2.  Mortgage payment hit on 9/2. It was paid, but we were charged $36 insufficient funds due solely to Funds Availability Policy.  After doing this first, Dollar then merrily hits us with two more $36 charges on smaller checks that would have cleared had they

Entering the prompt into ChatGPT, the performance is much better. Theme=customer service/banking/finance. 